In [2]:
import pandas as pd
import os

%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.insert(0, '..')
import os
os.chdir('..')

In [3]:
def load_data(
    youtuber: str = 'heyitsmindy',
    video_id: str = '9QSmr_2EGXM'
):

    with open(f'data/{youtuber}/{video_id}.txt', 'r') as f:
    
        transcript = f.read()

    parsed_data = []
    parsed_data.append(
        {"youtuber": youtuber, "video_id": f"{video_id}", "transcript": transcript}
    )
    # parsed_data = parse_transcript(transcript, youtuber, video_id)
    return parsed_data

parsed_data = load_data()
parsed_data[:2]


[{'youtuber': 'heyitsmindy',
  'video_id': '9QSmr_2EGXM',
  'transcript': '0:00 沒錯的話呢\n0:00 是他們之前的\n0:01 這個黑方氣墊的改版\n0:03 多了一點ㄉㄨㄞㄉㄨㄞ的觸感\n0:05 喔它的顯色度也太高了吧\n0:07 系列的唇釉啊\n0:08 比較偏像是那種\n0:09 霧面唇泥的妝感\n0:11 哈囉大家好我是Mindy\n0:12 前陣子我不是在IG上面\n0:14 上傳了兩支超級長的\n0:16 Olive Young購物分享的Reels嗎\n0:18 今天呢\n0:18 我想要在那之中挑一些\n0:20 大家可能會比較有興趣的產品\n0:22 來做一個也算是韓系彩妝新品\n0:24 Get Ready With Me\n0:25 然後今天會畫的應該\n0:26 就是一個我\n0:27 日常萬用\n0:28 無論是拍照啊什麼\n0:29 然後也是我之前去釜山\n0:31 穿韓服畫的那一種妝感\n0:33 那如果對今天的影片有興趣的人\n0:34 我們就趕快開始吧\n0:36 首先第一個呢\n0:37 我想要先用的是這個\n0:38 Aestura的精華水\n0:40 這個呢是我買他們的乳液\n0:41 送的小樣\n0:42 裡面試用到的\n0:44 它的質地有點像是牛奶精華水\n0:46 會比一般的化妝水再更滋潤一點\n0:49 我在美國的整段時間呢\n0:50 都是用這個當作是我妝前的第一步\n0:52 它的吸收速度還蠻快的\n0:54 然後我很喜歡它用完之後啊\n0:56 皮膚會有一點柔柔滑滑\n0:58 潤潤的感覺\n0:59 如果你是混合肌\n1:00 我感覺妝前擦一個這個就夠了\n1:02 你們看用完之後\n1:02 我的臉不會有那種很濕的感覺\n1:05 可是呢摸起來就是\n1:06 我覺得那個油水平衡\n1:07 有被調整的很好\n1:08 我現在整個皮膚摸起來就是\n1:10 軟軟彈彈的\n1:11 這個是我在Olive Young\n1:12 上面逛的時候\n1:13 有發現這個出現在他們\n1:14 美妝排行榜蠻前面\n1:16 好像第一名還是第二名吧\n1:17 是這個Yunjac的妝前\n1:20 後來我又陸陸續續看到一些\n1:21 彩

In [ ]:
from pydantic import BaseModel, Field
from typing import List

from enum import Enum
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
openai_client = OpenAI()


class VideoCategory(Enum):
    makeup = "makeup"
    skincare = "skincare"
    beauty = "beauty"

class Transcript(BaseModel):
    time: str = Field(description="The timestamp of the transcript segment.")
    text: str = Field(description="The text content of the transcript segment.")

class YTSummaryResponse(BaseModel):
    summary_zh: str = Field(description="The summary of the youtube video transcript.")
    title_zh: str = Field(description="The title of the youtube video title")
    summary_en: str = Field(description="English translation of the summary.")
    title_en: str = Field(description="English translation of the title.")
    category: VideoCategory = Field(description=(
        "Main beauty category of the video. Choose one of:\n"
        "- makeup: tutorials, reviews, product try-ons\n"
        "- skincare: routines, product reviews, ingredients\n"
        "- haircare: styling, coloring, or hair treatment tips"
    ))
    # transcripts: List[Transcript] = Field(description="The list of video transcripts.")


instructions = """
You are a multilingual beauty content analyst.

Your task is to analyze YouTube transcripts related to beauty, skincare, or makeup content.

Follow these steps carefully:

1. **Summarize** the full video transcript in **Chinese**, focusing on key talking points, tone, and product mentions.
2. **Translate** the summary and title into **English**.
3. **Categorize** the video into one of the following beauty categories:
   - makeup: tutorials, reviews, product try-ons
   - skincare: routines, product reviews, ingredients
   - haircare: styling, coloring, or hair treatment tips
""".strip()



def youtuber_summarize(user_prompt, output_format, model="gpt-4o-mini"):
    
    instructions = """
    You are a multilingual beauty content analyst.

    Your task is to analyze YouTube transcripts related to beauty, skincare, or makeup content.

    Follow these steps carefully:

    1. **Summarize** the full video transcript in **Chinese**, focusing on key talking points, tone, and product mentions.
    2. **Translate** the summary and title into **English**.
    3. **Categorize** the video into one of the following beauty categories:
    - makeup: tutorials, reviews, product try-ons
    - skincare: routines, product reviews, ingredients
    - haircare: styling, coloring, or hair treatment tips

    """.strip()

    messages = [
        {"role": "system", "content": instructions},
        {"role": "user", "content": user_prompt}
    ]

    response = openai_client.responses.parse(
        model=model,
        input=messages,
        text_format=output_format
    )

    return response.output[0].content[0].parsed



In [129]:
import re

def split_video_to_multiple_transcript(video):
    """
    Convert raw transcript string into list of {time, text} entries.
    """
    lines = video["transcript"].strip().split("\n")
    pattern = re.compile(r"(\d+:\d+)\s*(.*)")
    parsed = []

    transcript = ''
    for line in lines:
        match = pattern.match(line)
        if match:
            time_str, sentence = match.groups()
            # parsed.append({"youtuber": video["youtuber"], "video_id": video["video_id"], "time": time_str, "text": sentence.strip()})
            # eng = google_translate(sentence.strip())
            parsed.append({"time": time_str, "text": sentence.strip()})

            transcript += sentence.strip()
            transcript += ' '
    return parsed, transcript


In [130]:
split_transcript, transcript_str = split_video_to_multiple_transcript(parsed_data[0])
    

In [131]:
split_transcript

[{'time': '0:00', 'text': '沒錯的話呢'},
 {'time': '0:00', 'text': '是他們之前的'},
 {'time': '0:01', 'text': '這個黑方氣墊的改版'},
 {'time': '0:03', 'text': '多了一點ㄉㄨㄞㄉㄨㄞ的觸感'},
 {'time': '0:05', 'text': '喔它的顯色度也太高了吧'},
 {'time': '0:07', 'text': '系列的唇釉啊'},
 {'time': '0:08', 'text': '比較偏像是那種'},
 {'time': '0:09', 'text': '霧面唇泥的妝感'},
 {'time': '0:11', 'text': '哈囉大家好我是Mindy'},
 {'time': '0:12', 'text': '前陣子我不是在IG上面'},
 {'time': '0:14', 'text': '上傳了兩支超級長的'},
 {'time': '0:16', 'text': 'Olive Young購物分享的Reels嗎'},
 {'time': '0:18', 'text': '今天呢'},
 {'time': '0:18', 'text': '我想要在那之中挑一些'},
 {'time': '0:20', 'text': '大家可能會比較有興趣的產品'},
 {'time': '0:22', 'text': '來做一個也算是韓系彩妝新品'},
 {'time': '0:24', 'text': 'Get Ready With Me'},
 {'time': '0:25', 'text': '然後今天會畫的應該'},
 {'time': '0:26', 'text': '就是一個我'},
 {'time': '0:27', 'text': '日常萬用'},
 {'time': '0:28', 'text': '無論是拍照啊什麼'},
 {'time': '0:29', 'text': '然後也是我之前去釜山'},
 {'time': '0:31', 'text': '穿韓服畫的那一種妝感'},
 {'time': '0:33', 'text': '那如果對今天的影片有興趣的人'},
 {'time': '0:34', 'tex

In [132]:
split_transcript, transcript_str = split_video_to_multiple_transcript(parsed_data[0])
split_transcript[:5]

[{'time': '0:00', 'text': '沒錯的話呢'},
 {'time': '0:00', 'text': '是他們之前的'},
 {'time': '0:01', 'text': '這個黑方氣墊的改版'},
 {'time': '0:03', 'text': '多了一點ㄉㄨㄞㄉㄨㄞ的觸感'},
 {'time': '0:05', 'text': '喔它的顯色度也太高了吧'}]

In [133]:

    
# summary
video_summary = youtuber_summarize(
    # instructions=instructions,
    user_prompt=str(split_transcript),
    output_format=YTSummaryResponse
)

In [134]:
video_summary

YTSummaryResponse(summary_zh='這段影片由Mindy主講，重點是韓系彩妝新品的介紹與實際示範。她首先提到了一個改版的黑方氣墊，顯色度非常高，接著分享了她在IG推廣的一些產品。影片的主要內容是「Get Ready With Me」的妝容示範，使用了Aestura的精華水和Yunjac妝前乳，這些產品都在美妝排行榜上名列前茅。Mindy也將兩種妝前乳進行比較，表達了各自的特點與效果。接下來，她使用了CLIO的新氣墊，分享了這款氣墊與舊版的差異，並且提到遮瑕膏和眼影的多款選擇，其中包括CLIO的聯名眼影盤和Etude的口袋眼影盤。影片最後，她還分享了幾支唇釉的使用體驗。整個妝容強調了和諧的眼、頰、唇搭配，以及在加強妝感時不過於誇張的細節。', title_zh='韓系彩妝新品實測｜與我一起化妝！', summary_en="In this video, Mindy shares her experience with new Korean makeup products and demonstrates a 'Get Ready With Me' makeup look. She discusses a revamped cushion foundation known for its high pigment and introduces some popular products from her previous IG posts. Throughout the video, she highlights key items like Aestura's essence and Yunjac's primer while comparing their effects. She uses CLIO's new cushion foundation, discussing its differences from the older version and showcasing various concealers and eyeshadow options, including CLIO's collaborative eyeshadow palettes and Etude's pocket eyeshadow palette. The

In [11]:
def sliding_window(
        video_id, 
        youtuber, 
        parsed, 
        video_summary, 
        window_size=10, 
        step_size=5):
    """
    Create overlapping chunks from a list of transcript dicts.

    Args:
        parsed (list[dict]): Transcript entries with 'text', 'time', etc.
        video_summary: Object with title_en, summary_en, category.
        window_size (int): Number of transcript entries per chunk.
        step_size (int): How many entries to move forward (creates overlap).

    Returns:
        list[dict]: Overlapping transcript chunks.
    """
    chunks = []
    # video_id = parsed[0]['video_id']
    # youtuber = parsed[0]['youtuber']
    total = len(parsed)
    chunk_id = 1

    for start in range(0, total, step_size):
        end = min(start + window_size, total)
        window = parsed[start:end]
        
        # If window too small at end, you can skip or include:
        if len(window) < 2:
            break  # stop if not enough data left

        # Combine texts from the window
        # text_chunk = " ".join([entry.text for entry in window])
        text_chunk = " ".join([entry['text'] for entry in window])
        # original_text_chunk = " ".join([entry.original_text for entry in window])

        chunks.append({
            'video_id': video_id,
            'youtuber': youtuber,
            'title': video_summary.title_en,
            'summary': video_summary.summary_en,
            'category': video_summary.category.value,
            'title_zh': video_summary.title_zh,
            'summary_zh': video_summary.summary_zh,
            'chunk_id': chunk_id,
            'start_time': window[0]['time'],
            'end_time': window[-1]['time'],
            'content': text_chunk,
            # 'original_content': original_text_chunk
        })

        chunk_id += 1

        if end == total:  # reached end of list
            break

    return chunks

In [12]:
chunks = sliding_window(
        video_id=parsed_data[0]['video_id'],
        youtuber=parsed_data[0]['youtuber'],
        parsed=split_transcript,
        video_summary=video_summary,
        window_size=15,
        step_size=3
    )

In [13]:
chunks

[{'video_id': '9QSmr_2EGXM',
  'youtuber': 'heyitsmindy',
  'title': 'Korean Makeup New Products Review',
  'summary': "In this video, Mindy introduces several new Korean makeup products and shares her experiences and looks created with them. She starts with Aestura's essence water, noting its fast absorption and moisturizing properties suitable for combination skin. She then discusses primer products, comparing Yunjac and dasique in terms of texture. Mindy also tries the CLIO black cushion, comparing it with its predecessor and concluding that their coverage and finish are quite similar. Next, she reviews well-known concealers and eyeshadows, deeply contrasting old and new products based on pigmentation, texture, and durability. The video's focus is on the practical effects of the products and personal impressions, alongside tips for makeup application. Mindy encourages viewers to like and subscribe at the end.",
  'category': 'makeup',
  'title_zh': '韓系彩妝新品心得分享',
  'summary_zh': '在這段

In [137]:
# Test functino
from service.youtube_extraction.src.youtube_extraction.youtube_process import YoutuberTranscriptProcessor

ytp = YoutuberTranscriptProcessor(youtuber='heyitsmindy')


In [ ]:
ytp.load_transcript('https://www.youtube.com/watch?v=2SvN45DKWFg&sttick=0')

[{'video_id': '2SvN45DKWFg',
  'youtuber': 'heyitsmindy',
  'title': 'Sephora Summary: What to Buy and What Not to!',
  'summary': "In this video, Mindy shares various products she has purchased from Sephora over the past few years, including lip balms, primers, foundations, concealers, setting powders, blushes, and lip colors. She emphasizes the use and effects of several products, especially recommending the Topicals lip balm, Tower 28's SOS primer spray, and Natasha Denona's Hy-Glam foundation, believing these products are suitable for Asian skin tones and worth purchasing during sales. The video also mentions products she does not recommend, such as Charlotte Tilbury's Airbrush foundation and certain brands of blushes, highlighting their shortcomings in coverage and finish. At the end of the video, Mindy hopes viewers find it helpful and looks forward to their feedback.",
  'category': 'makeup',
  'title_zh': 'Sephora 大總結：買什麼、買什麼不值得！',
  'summary_zh': '在這支影片中，Mindy 分享了她過去幾年在 Sephor

In [42]:
# Minsearch

from minsearch import Index
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import jieba

/Users/yenchunchen/Desktop/Project/health-agent/.venv/lib/python3.12/site-packages/jieba/__init__.py:44: SyntaxWarning: invalid escape sequence '\.'
  re_han_default = re.compile("([\u4E00-\u9FD5a-zA-Z0-9+#&\._%\-]+)", re.U)
/Users/yenchunchen/Desktop/Project/health-agent/.venv/lib/python3.12/site-packages/jieba/__init__.py:46: SyntaxWarning: invalid escape sequence '\s'
  re_skip_default = re.compile("(\r\n|\s)", re.U)
/Users/yenchunchen/Desktop/Project/health-agent/.venv/lib/python3.12/site-packages/jieba/finalseg/__init__.py:78: SyntaxWarning: invalid escape sequence '\.'
  re_skip = re.compile("([a-zA-Z0-9]+(?:\.\d+)?%?)")


In [197]:
from service.youtube_extraction.src.youtube_extraction.youtube_search import YoutuberTranscriptSearcher


yts = YoutuberTranscriptSearcher(youtuber='heyitsmindy')



In [199]:
yts.check_index_es(video_id='9QSmr_2EGXM')

Processing... 9QSmr_2EGXM


100%|██████████| 466/466 [00:14<00:00, 32.03it/s]


Added new chunk to elasticsearch!


In [ ]:
'/Users/yenchunchen/Desktop/Project/health-agent/.cache/heyitsmindy_15_3_Chinese/heyitsmindy_9QSmr_2EGXM.json'
'/Users/yenchunchen/Desktop/Project/health-agent/.cache/heyitsmindy_15_3_Chinese/heyitsmindy_9QSmr_2EGXM.json'

In [175]:
import os
print(os.getcwd())

/Users/yenchunchen/Desktop/Project/health-agent


In [55]:

# embedding
from sentence_transformers import SentenceTransformer

/Users/yenchunchen/Desktop/Project/health-agent/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [56]:
import numpy as np

model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

In [94]:
from tqdm import tqdm
for chunk in tqdm(chunks):
    # text_to_embed = f"{chunk['title']} {chunk['summary']} {chunk['title_zh']} {chunk['summary_zh']} {chunk['content']}"
    text_to_embed = f"{chunk['title_zh']} {chunk['summary_zh']} {chunk['content']}"
    embedding = model.encode(text_to_embed).tolist()
    chunk['embedding'] = embedding
    # break

100%|██████████| 466/466 [00:08<00:00, 54.39it/s]


In [ ]:
from service.youtube_extraction.src.youtube_extraction.youtube_search import YoutuberTranscriptSearcher


yts = YoutuberTranscriptSearcher()


In [187]:
# Delete the index, ignoring 400 (bad request) and 404 (not found) errors
es.indices.delete(index=index_name, ignore=[400, 404])

/var/folders/t7/tf0dc46s4hdc5zgvcgk2vhd80000gn/T/ipykernel_94731/2382779415.py:2: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.delete(index=index_name, ignore=[400, 404])


ObjectApiResponse({'acknowledged': True})

In [97]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")

# index_name = "youtube"
index_name = "youtube_bilingual"
index_body = {
    "mappings": {
        "properties": {
            "video_id": {"type": "keyword"},
            "youtuber": {"type": "keyword"},
            "category": {"type": "keyword"},
            "title": {"type": "text"},
            "summary": {"type": "text"},
            "title_zh": {"type": "text"},
            "summary_zh": {"type": "text"},
            "content": {"type": "text"},
            "chunk_id": {"type": "integer"},
            "start_time": {"type": "keyword"},
            "end_time": {"type": "keyword"},
            "embedding": {"type": "dense_vector", "dims": 384, "index": True, "similarity": "cosine"}
        }
    }
}
# Delete the index, ignoring 400 (bad request) and 404 (not found) errors
es.indices.delete(index=index_name, ignore=[400, 404])

es.indices.create(index=index_name, body=index_body, ignore=400)



/var/folders/t7/tf0dc46s4hdc5zgvcgk2vhd80000gn/T/ipykernel_94731/3489381563.py:26: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.delete(index=index_name, ignore=[400, 404])
/var/folders/t7/tf0dc46s4hdc5zgvcgk2vhd80000gn/T/ipykernel_94731/3489381563.py:28: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.create(index=index_name, body=index_body, ignore=400)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'youtube_bilingual'})

In [116]:
for chunk in tqdm(chunks):
    es.index(index=index_name, document=chunk)

100%|██████████| 466/466 [00:04<00:00, 109.78it/s]


In [199]:
es.indices.refresh(index=index_name)


TypeError: isinstance() arg 2 must be a type, a tuple of types, or a union

In [112]:
query_text = "必買的前三名"  # Could be English or Chinese
query_emb = model.encode(query_text).tolist()

response = es.search(
    index=index_name,
    knn={
        "field": "embedding",
        "query_vector": query_emb,
        "k": 10,
        "num_candidates": 10
    }
)

for hit in response['hits']['hits']:
    print(hit['_score'])
    print(hit['_source']['title'])
    print(hit['_source']['content'])

0.61984
Korean Makeup New Products Review
首先第一個呢想跟大家分享的 就是這個Etude的霧面唇釉 那我這邊有的色號呢 是9號、11號跟12號 如果我記得沒錯的話 這三個新色 是跟IG上面一個很有名的化妝師 共同聯名開發的 然後那個彩妝師她自己喜歡的妝感 都會我覺得再更有個性一點 所以這三個顏色都是屬於那種 低飽和很秋冬的顏色很好看 那這三個顏色裡面 我用的最兇的就是這個9號色 首先呢我很喜歡它這種
0.61984
Korean Makeup New Products Review
是9號、11號跟12號 如果我記得沒錯的話 這三個新色 是跟IG上面一個很有名的化妝師 共同聯名開發的 然後那個彩妝師她自己喜歡的妝感 都會我覺得再更有個性一點 所以這三個顏色都是屬於那種 低飽和很秋冬的顏色很好看 那這三個顏色裡面 我用的最兇的就是這個9號色 首先呢我很喜歡它這種 胖胖小小的外包裝 我覺得很可愛 再來就是它裡面的這個刷頭啊
0.61984
Korean Makeup New Products Review
是跟IG上面一個很有名的化妝師 共同聯名開發的 然後那個彩妝師她自己喜歡的妝感 都會我覺得再更有個性一點 所以這三個顏色都是屬於那種 低飽和很秋冬的顏色很好看 那這三個顏色裡面 我用的最兇的就是這個9號色 首先呢我很喜歡它這種 胖胖小小的外包裝 我覺得很可愛 再來就是它裡面的這個刷頭啊 也非常的好用 我個人是很喜歡這種 小小胖胖像是小枕頭一樣的刷頭
0.61984
Korean Makeup New Products Review
都會我覺得再更有個性一點 所以這三個顏色都是屬於那種 低飽和很秋冬的顏色很好看 那這三個顏色裡面 我用的最兇的就是這個9號色 首先呢我很喜歡它這種 胖胖小小的外包裝 我覺得很可愛 再來就是它裡面的這個刷頭啊 也非常的好用 我個人是很喜歡這種 小小胖胖像是小枕頭一樣的刷頭 然後這個我的用法呢 就是我會拿它這個尖端的地方 薄薄的上一層在我的唇緣
0.61984
Korean Makeup New Products Review
那這三個顏色裡面 我用的最兇的就是這個9號色 首先呢我很喜歡它這種 胖胖小小的外包裝 我覺得很可愛 再來就是它裡面的這個刷頭啊 也非常的好用 我個人是很喜歡這種 小小胖胖像是小枕

In [99]:
query_text = "遮瑕度好的產品"  # Could be English or Chinese
query_emb = model.encode(query_text).tolist()

response = es.search(
    index=index_name,
    knn={
        "field": "embedding",
        "query_vector": query_emb,
        "k": 5,
        "num_candidates": 10
    }
)

for hit in response['hits']['hits']:
    print(hit['_score'])
    print(hit['_source']['title'])
    print(hit['_source']['content'])

0.7776357
Korean Makeup New Products Review
沒錯的話呢 是他們之前的 這個黑方氣墊的改版 多了一點ㄉㄨㄞㄉㄨㄞ的觸感 喔它的顯色度也太高了吧 系列的唇釉啊 比較偏像是那種 霧面唇泥的妝感 哈囉大家好我是Mindy 前陣子我不是在IG上面 上傳了兩支超級長的 Olive Young購物分享的Reels嗎 今天呢 我想要在那之中挑一些 大家可能會比較有興趣的產品
0.7776357
Korean Makeup New Products Review
多了一點ㄉㄨㄞㄉㄨㄞ的觸感 喔它的顯色度也太高了吧 系列的唇釉啊 比較偏像是那種 霧面唇泥的妝感 哈囉大家好我是Mindy 前陣子我不是在IG上面 上傳了兩支超級長的 Olive Young購物分享的Reels嗎 今天呢 我想要在那之中挑一些 大家可能會比較有興趣的產品 來做一個也算是韓系彩妝新品 Get Ready With Me 然後今天會畫的應該
0.7776357
Korean Makeup New Products Review
比較偏像是那種 霧面唇泥的妝感 哈囉大家好我是Mindy 前陣子我不是在IG上面 上傳了兩支超級長的 Olive Young購物分享的Reels嗎 今天呢 我想要在那之中挑一些 大家可能會比較有興趣的產品 來做一個也算是韓系彩妝新品 Get Ready With Me 然後今天會畫的應該 就是一個我 日常萬用 無論是拍照啊什麼
0.7776357
Korean Makeup New Products Review
前陣子我不是在IG上面 上傳了兩支超級長的 Olive Young購物分享的Reels嗎 今天呢 我想要在那之中挑一些 大家可能會比較有興趣的產品 來做一個也算是韓系彩妝新品 Get Ready With Me 然後今天會畫的應該 就是一個我 日常萬用 無論是拍照啊什麼 然後也是我之前去釜山 穿韓服畫的那一種妝感 那如果對今天的影片有興趣的人
0.7776357
Korean Makeup New Products Review
今天呢 我想要在那之中挑一些 大家可能會比較有興趣的產品 來做一個也算是韓系彩妝新品 Get Ready With Me 然後今天會畫的應該 就是一個我 日常萬用 無論是拍照啊什麼 然後也是我之前去釜山 穿韓服

In [102]:
# ############## Elastic Search 
from elasticsearch import Elasticsearch
es_client = Elasticsearch(
    "http://localhost:9200",  # adjust if needed
    request_timeout=120        # increase timeout
)

In [31]:

print(es_client.info())

{'name': 'd9a91a4f0006', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'hAYzzaUsRMmESj4kE9VQMg', 'version': {'number': '9.1.1', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '5e94055934defa56e454868b7783b2a3b683785e', 'build_date': '2025-08-05T01:07:31.959947279Z', 'build_snapshot': False, 'lucene_version': '10.2.2', 'minimum_wire_compatibility_version': '8.19.0', 'minimum_index_compatibility_version': '8.0.0'}, 'tagline': 'You Know, for Search'}


In [32]:
# Define the index name
index_name = "youtube_transcripts"


index_body = {
    # "settings": {
    #     "analysis": {
    #         "analyzer": {
    #             # English analyzer
    #             "english_analyzer": {
    #                 "type": "standard"
    #             },
    #             # Chinese analyzer using IK Max Word
    #             "chinese_analyzer": {
    #                 "type": "ik_max_word"
    #             }
    #         }
    #     }
    # },
    "mappings": {
        "properties": {
            "video_id": {"type": "keyword"},
            "youtuber": {"type": "keyword"},
            # "title": {"type": "text", "analyzer": "english_analyzer"},
            # "summary": {"type": "text", "analyzer": "english_analyzer"},
            # "category": {"type": "keyword"},
            # "chunk_id": {"type": "integer"},
            # "start_time": {"type": "keyword"},
            # "end_time": {"type": "keyword"},
            # "content": {
            #     "type": "text",
            #     "fields": {
            #         # separate field for Chinese content using IK Analyzer
            #         "chinese": {"type": "text", "analyzer": "chinese_analyzer"}
            #     }
            # }
        }
    }
}

In [33]:
# Delete the index, ignoring 400 (bad request) and 404 (not found) errors
es_client.indices.delete(index=index_name, ignore=[400, 404])

/var/folders/t7/tf0dc46s4hdc5zgvcgk2vhd80000gn/T/ipykernel_94731/4095105296.py:2: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es_client.indices.delete(index=index_name, ignore=[400, 404])


ObjectApiResponse({'acknowledged': True})

In [34]:
# Create the index
if not es_client.indices.exists(index=index_name):
    es_client.indices.create(index=index_name, body=index_body)
    print(f"Index '{index_name}' created successfully.")
else:
    print(f"Index '{index_name}' already exists.")

Index 'youtube_transcripts' created successfully.


In [44]:
from tqdm.auto import tqdm

In [28]:
chunks[0]

{'video_id': '9QSmr_2EGXM',
 'youtuber': 'heyitsmindy',
 'title': 'Korean Makeup New Products Review',
 'summary': "In this video, Mindy introduces several new Korean makeup products and shares her experiences and looks created with them. She starts with Aestura's essence water, noting its fast absorption and moisturizing properties suitable for combination skin. She then discusses primer products, comparing Yunjac and dasique in terms of texture. Mindy also tries the CLIO black cushion, comparing it with its predecessor and concluding that their coverage and finish are quite similar. Next, she reviews well-known concealers and eyeshadows, deeply contrasting old and new products based on pigmentation, texture, and durability. The video's focus is on the practical effects of the products and personal impressions, alongside tips for makeup application. Mindy encourages viewers to like and subscribe at the end.",
 'category': 'makeup',
 'title_zh': '韓系彩妝新品心得分享',
 'summary_zh': '在這段影片中，Min

In [60]:
def elastic_search(query, num_results=15):
    es_query = {
        "size": num_results,
        "query": {
            "multi_match": {
                "query": query,
                "type": "best_fields",
                "fields": ["title", "summary", "category"],
            }
        }
    }

    response = es_client.search(index=index_name, body=es_query)

    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [61]:
search_results = elastic_search('Best foundation')

ApiError: ApiError(503, 'search_phase_execution_exception', None)

In [15]:
from service.youtube_extraction.src.youtube_extraction.youtube_search  import YoutuberTranscriptSearcher

In [23]:
from elasticsearch import Elasticsearch

In [24]:
# Delete the index, ignoring 400 (bad request) and 404 (not found) errors
es_client = Elasticsearch("http://localhost:9200")
es_client.indices.delete(index='youtube_bilingual', ignore=[400, 404])

/var/folders/t7/tf0dc46s4hdc5zgvcgk2vhd80000gn/T/ipykernel_77998/2194080289.py:3: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es_client.indices.delete(index='youtube_bilingual', ignore=[400, 404])


ObjectApiResponse({'acknowledged': True})

In [30]:
yts = YoutuberTranscriptSearcher()

In [ ]:
question = "What foundations would you recommend from Korea?"
question = "Natasha Denona"
question = "必買的 Sephora 遮瑕 用來用去還是Hourglass"
question = "Hourglass 遮瑕系列"
# question = "What are the must-buy concealers at Sephora?"
# question = "就是Natasha Denona 新出的這個Hy-Glam粉底液"

In [45]:
yts.search_es(question)

[{'score': 0.8584933,
  'video_id': '2SvN45DKWFg',
  'title': 'Sephora Summary: What to Buy and What Not to!',
  'start_time': '40:02',
  'content': '讓我的唇周的顏色整理得很乾淨 那除了這2個打底色之外呢 我還有買了這個是比較偏粉的 然後這個是霧面的質地 這個顏色是Peony 然後還有這個顏色我比較常用 這個顏色是Blush Brown 它就是這種很日常很百搭的裸棕色 再來唇膏呢我還很喜歡 這個Dior的霧面唇膏 我很喜歡的這2個顏色是 100號跟220 100號是我之前有跟大家推薦過的 它是一個非常百搭 又很貴氣'},
 {'score': 0.8575056,
  'video_id': '2SvN45DKWFg',
  'title': 'Sephora Summary: What to Buy and What Not to!',
  'start_time': '46:00',
  'content': '是比較扁平的 所以如果你不喜歡 唇蜜上的太多的人 你應該會比較喜歡這個刷頭 然後比較深的這個顏色 我也覺得還蠻不錯的 這個顏色是Milky Mocha 好最後呢我來提一個 不是彩妝品 但是應該也會有人有興趣的 就是這個Rare Beauty的護手霜 這個護手霜就是還不錯啦 如果你想要買來送人 或是有你缺一個護手霜 或是你喜歡這個包裝的話'},
 {'score': 0.8554792,
  'video_id': '2SvN45DKWFg',
  'title': 'Sephora Summary: What to Buy and What Not to!',
  'start_time': '44:33',
  'content': '都是我覺得很OK的 不覺得就很適合買來送人嗎 就連它背面的這個 包裝設計也很好看 再來呢這個也是我超級喜歡的 唇蜜質地 這個是REFY的唇蜜 那我很喜歡REFY的這個唇蜜 就是因為它的鏡面感 非常的強 所以擦在嘴巴上就很好看 它那個光點會很集中 那我買的這5個顏色裡面 我自己比較喜歡的是 這個顏色Taupe'},
 {'score': 0.8541172,
  'video